In [30]:
##17 hashes are duplicated i.e, have duplicate URLs and have been downled as a same report with same TTPs as two different hashes -> 120 - 17 is the true ground truth
# Function to find unique hashes with URLs and track removed hashes
import json
import os

In [31]:
# Function to count the number of URLs in the file
def count_urls(file_path):
    url_count = 0
    
    # Open and read the file
    with open(file_path, 'r') as file:
        for line in file:
            # Count non-empty lines (assuming each line contains one URL)
            if line.strip():  # Strip any whitespace and check if the line is not empty
                url_count += 1
                
    return url_count

file_path = r"C:\Users\ricewater\Documents\CTIDownloads\downloads\downloads\20240930_urls.txt"

# Call the function and print the result
print(f'Total number of URLs: {count_urls(file_path)}')

Total number of URLs: 1372


In [32]:
import json

def read_jsonl_to_dict(file_path):
    """
    Reads a JSONL file and converts it into a list of dictionaries.

    Parameters
    ----------
    file_path : str
        The path to the JSONL file.

    Returns
    -------
    list of dict
        A list of dictionaries where each dictionary represents one line in the JSONL file.
    """
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Parse each line into a dictionary and append to the list
            data.append(json.loads(line))
    
    return data


In [33]:
from collections import defaultdict

def find_duplicate_urls(merged_data):
    """
    Identifies entries with duplicate URLs and returns their associated download_sha256 values.

    Parameters
    ----------
    merged_data : list of dict
        The merged JSONL data.

    Returns
    -------
    dict
        A dictionary where each key is a URL and the value is a list of download_sha256
        values for entries with the same URL.
    """
    # Dictionary to store URLs as keys and lists of download_sha256 as values
    url_to_sha256 = defaultdict(list)
    
    # Iterate through all entries in the merged data
    for entry in merged_data:
        url = entry.get('url')  # Safely get the 'url', if it exists
        download_sha256 = entry.get('download_sha256')  # Safely get the 'download_sha256'
        
        # Only proceed if both url and download_sha256 are present
        if url and download_sha256:
            # Add the download_sha256 to the list corresponding to the URL
            url_to_sha256[url].append(download_sha256)
    
    # Filter to keep only URLs with more than one associated download_sha256 (duplicate URLs)
    duplicate_urls = {url: sha256s for url, sha256s in url_to_sha256.items() if len(sha256s) > 1}
    
    return duplicate_urls


# Find and print duplicate URLs and their download_sha256 values
duplicate_urls = find_duplicate_urls(data)

if duplicate_urls:
    #print("Entries with duplicate URLs:")
    print(f"Total entries with duplicate URLs: {len(duplicate_urls)}")
    #for url, sha256s in duplicate_urls.items():
     #   print(f"URL: {url}")
     #   print(f"Download SHA256 values: {sha256s}\n")
else:
    print("No duplicate URLs found.")


Total entries with duplicate URLs: 395


In [34]:
# Example usage:
file_path = r"C:\Users\ricewater\Documents\CTIDownloads\downloads\downloads\20241008_downloads.jsonl"
data = read_jsonl_to_dict(file_path)
print(f"Total MITRE JSONL entries: {len(data)}")

Total MITRE JSONL entries: 1392


In [35]:
import os

def compare_files_content(file_path1, file_path2):
    """
    Compare the contents of two files to see if they are identical.
    
    Parameters
    ----------
    file_path1 : str
        Path to the first file.
    file_path2 : str
        Path to the second file.
    
    Returns
    -------
    bool
        True if the files have identical content, False otherwise.
    """
    try:
        with open(file_path1, 'rb') as file1, open(file_path2, 'rb') as file2:
            # Compare the files byte by byte
            return file1.read() == file2.read()
    except FileNotFoundError:
        print(f"File not found: {file_path1} or {file_path2}")
        return False

def check_duplicate_files_content(folder_path, duplicate_urls):
    """
    Check if the downloaded files corresponding to the given SHA256 hashes have the same content.
    
    Parameters
    ----------
    folder_path : str
        The primary folder containing the downloaded files.
    duplicate_urls : dict
        A dictionary with URLs as keys and lists of SHA256 hashes as values.
    
    Returns
    -------
    None
    """
    identical_files_count = 0
    
    for url, sha256_list in duplicate_urls.items():
        #print(f"Checking duplicate files for URL: {url}")
        
        matched_files = []
   
        
        # Loop through the folder and find files that match the SHA256 hashes
        for sha256_hash in sha256_list:
            file_name = f"{sha256_hash}.download"  # Assuming the file name is based on the hash
            file_path_primary = os.path.join(folder_path, file_name)
            
            # Check if the file exists in the primary folder
            if os.path.exists(file_path_primary):
                matched_files.append(file_path_primary)

            else:
                print(f"File {file_name} not found in the folder i.e., not downloaded")
        
        # If there are exactly two matched files, compare their content
        if len(matched_files) == 2:
            file1, file2 = matched_files
            #print(f"Comparing {file1} and {file2}")
            if compare_files_content(file1, file2):
                identical_files_count += 1
                ##print(f"Files {file1} and {file2} have identical content.")
            #else:
            #    print(f"Files {file1} and {file2} have different content.")
        else:
            print(f"Could not find exactly two matching files or more that two files for {url}.")



    
    print(f"\nTotal identical files found: {identical_files_count}")

# Example usage:
folder_path = r"C:\Users\ricewater\Documents\CTIDownloads\downloads\downloads\20241008_downloads"

check_duplicate_files_content(folder_path, duplicate_urls)



File 0252a19e0a9ba187823828530c0117a4be0a99717ff643c72f54b8db09a63b68.download not found in the folder i.e., not downloaded
File 0252a19e0a9ba187823828530c0117a4be0a99717ff643c72f54b8db09a63b68.download not found in the folder i.e., not downloaded
Could not find exactly two matching files or more that two files for http://cdn0.vox-cdn.com/assets/4589853/crowdstrike-intelligence-report-putter-panda.original.pdf.
Could not find exactly two matching files or more that two files for https://www.mandiant.com/sites/default/files/2022-02/rt-apt41-dual-operation.pdf.
Could not find exactly two matching files or more that two files for https://www.secureworks.com/research/threat-profiles/iron-viking.
Could not find exactly two matching files or more that two files for https://blog.talosintelligence.com/2020/03/bisonal-10-years-of-play.html.

Total identical files found: 266


In [36]:
import os

def count_files_in_folders(folder_path):
    """
    Count the number of files in the given folder paths.

    Parameters
    ----------
    folder_path : str
        The primary folder containing the files.

    Returns
    -------
    int
        The total count of files in both folders.
    """
    # Count files in the primary folder
    primary_folder_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]


    # Calculate the total count of files 
    total_files_count = len(primary_folder_files) 

    print(f"Total number of files in both folders: {total_files_count}")

    return total_files_count

# Example usage:

#folder_path = r"C:\Users\ricewater\Documents\CTIDownloads\downloads\downloads\20241008_downloads"
# Get total file count
total_files = count_files_in_folders(folder_path)


Total number of files in both folders: 900


In [37]:
iocs_path = r"C:\Users\ricewater\Documents\CTIDownloads\downloads\downloads\iocs2"
# Get total file count
total_files = count_files_in_folders(iocs_path)

Total number of files in both folders: 120


In [38]:
import os
import re

def count_cumulative_unique_ttps(folder_path):
    """
    Count cumulative unique TTP IDs across all files in the given folder paths.
    
    Parameters
    ----------
    folder_path : str
        Path to the first folder.
    secondary_folder_path : str
        Path to the second folder.
    
    Returns
    -------
    set
        A set of unique TTP IDs found in both folders.
    """
    unique_ttps = set()

    # Regex pattern to capture full TTP IDs (e.g., T1003, T1003.001)
    ttp_pattern = re.compile(r'\bT\d{4}(?:\.\d+)?\b')

    # Loop through both folders
    for folder in [folder_path]:
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)

            # Skip hidden or backup files
            if file_name.startswith('_'):
                print(f"Skipping hidden or backup file: {file_name}")
                continue

            if os.path.isfile(file_path):
                #print(f"Reading TTPs from file: {file_name}")
                try:
                    # Open the file and check each line for TTP patterns
                    with open(file_path, 'r', errors='ignore') as file:
                        for line in file:
                            # Find all TTP IDs in the line and add them directly to the cumulative set
                            matches = ttp_pattern.findall(line)
                            unique_ttps.update(matches)  # Add each unique TTP ID found in the line
                except Exception as e:
                    print(f"Error reading file {file_name}: {e}")
    
    return unique_ttps

# Example usage
folder_path = r"C:\Users\ricewater\Documents\CTIDownloads\downloads\downloads\iocs2"


unique_ttps = count_cumulative_unique_ttps(folder_path)

# Display the unique TTPs
print(f"Cumulative Unique TTPs found: {len(unique_ttps)}")
#for ttp in unique_ttps:
#   print(ttp)


Cumulative Unique TTPs found: 461
